In [2]:
%%html
<style>
figure {
    display: flex;
    flex-direction: column;
    align-items: center;
}
figure img {
    max-width: 900px;
    width: 80%;
    margin-bottom: 2em;
}
figcaption {
    aria-hidden: true;
}
</style>

# Markov Chain Monte Carlo and Emergent Disorder

The results for the phase diagram were obtained with a classical MCMC method which we discuss in the following. It allows us to solve our long-range FK model efficiently, yielding unbiased estimates of thermal expectation values and linking it to disorder physics in a translationally invariant setting.

Since the spin configurations are classical, the Hamiltonian can be split into a classical spin part $H_s$ and an operator valued part $H_c$. $$\begin{aligned}
H_s& = - \frac{U}{2}S_i + \sum_{i, j}^{N} J_{ij} S_i S_j \\
H_c& = \sum_i U S_i c^\dag_{i}c_{i} -t(c^\dag_{i}c_{i+1} + c^\dag_{i+1}c_{i}) \end{aligned}$$ The partition function can then be written as a sum over spin configurations, $\vec{S} = (S_0, S_1...S_{N-1})$: $$\begin{aligned}
\Z = \Tr e^{-\beta H}= \sum_{\vec{S}} e^{-\beta H_s} \Tr_c e^{-\beta H_c} .\end{aligned}$$ The contribution of $H_c$ to the grand canonical partition function can be obtained by performing the sum over eigenstate occupation numbers giving $-\beta F_c[\vec{S}] = \sum_k \ln{(1 + e^{- \beta \epsilon_k})}$ where ${\epsilon_k[\vec{S}]}$ are the eigenvalues of the matrix representation of $H_c$ determined through exact diagonalisation. This gives a partition function containing a classical energy which corresponds to the long-range interaction of the spins, and a free energy which corresponds to the quantum subsystem. $$\begin{aligned}
\Z = \sum_{\vec{S}} e^{-\beta H_S[\vec{S}] - \beta F_c[\vec{S}]} = \sum_{\vec{S}} e^{-\beta E[\vec{S}]}\end{aligned}$$

MCMC defines a weighted random walk over the spin states $(\vec{S}_0, \vec{S}_1, \vec{S}_2, ...)$, such that the likelihood of visiting a particular state converges to its Boltzmann probability $p(\vec{S}) = \Z^{-1} e^{-\beta E}$ [@binderGuidePracticalWork1988; @kerteszAdvancesComputerSimulation1998; @wolffMonteCarloErrors2004]. Hence, any observable can be estimated as a mean over the states visited by the walk. $$\begin{aligned}
 \label{eq:thermal_expectation}
\tex{O}& = \sum_{\vec{S}} p(\vec{S}) \tex{O}_{\vec{S}} = \sum_{i = 0}^{M} \tex{O}_{\vec{S}_i} + \mathcal{O}(\tfrac{1}{\sqrt{M}})\\
\tex{O}_{\vec{S}}& = \sum_{\nu} n_F(\epsilon_{\nu}) \expval{O}{\nu}\end{aligned}$$ Where $\nu$ runs over the eigenstates of $H_c$ for a particular spin configuration and $n_F(\epsilon) = \left(e^{-\beta\epsilon} + 1\right)^{-1}$ is the Fermi function.

The choice of the transition function for MCMC is under-determined as one only needs to satisfy a set of balance conditions for which there are many solutions [@kellyReversibilityStochasticNetworks1981]. Here, we incorporate a modification to the standard Metropolis-Hastings algorithm [@hastingsMonteCarloSampling1970] gleaned from Krauth [@krauthIntroductionMonteCarlo1998]. Let us first recall the standard algorithm which decomposes the transition probability into $\T(a \to b) = \p(a \to b)\A(a \to b)$. Here, $\p$ is the proposal distribution that we can directly sample from while $\A$ is the acceptance probability. The standard Metropolis-Hastings choice is $$\A(a \to b) = \min\left(1, \frac{\p(b\to a)}{\p(a\to b)} e^{-\beta \Delta E}\right)\;,$$ with $\Delta E = E_b - E_a$. The walk then proceeds by sampling a state $b$ from $\p$ and moving to $b$ with probability $\A(a \to b)$. The latter operation is typically implemented by performing a transition if a uniform random sample from the unit interval is less than $\A(a \to b)$ and otherwise repeating the current state as the next step in the random walk. The proposal distribution is often symmetric so does not appear in $\A$. Here, we flip a small number of sites in $b$ at random to generate proposals, which is indeed symmetric.

In our computations [@hodsonMCMCFKModel2021] we employ a modification of the algorithm which is based on the observation that the free energy of the FK system is composed of a classical part which is much quicker to compute than the quantum part. Hence, we can obtain a computational speedup by first considering the value of the classical energy difference $\Delta H_s$ and rejecting the transition if the former is too high. We only compute the quantum energy difference $\Delta F_c$ if the transition is accepted. We then perform a second rejection sampling step based upon it. This corresponds to two nested comparisons with the majority of the work only occurring if the first test passes and has the acceptance function $$\A(a \to b) = \min\left(1, e^{-\beta \Delta H_s}\right)\min\left(1, e^{-\beta \Delta F_c}\right)\;.$$

See Appendix [\[app:balance\]](#app:balance){reference-type="ref" reference="app:balance"} for a proof that this satisfies the detailed balance condition.

For the model parameters used in Fig. [2](#fig:indiv_IPR){reference-type="ref" reference="fig:indiv_IPR"}, we find that with our new scheme the matrix diagonalisation is skipped around 30% of the time at $T = 2.5$ and up to 80% at $T = 1.5$. We observe that for $N = 50$, the matrix diagonalisation, if it occurs, occupies around 60% of the total computation time for a single step. This rises to 90% at N = 300 and further increases for larger N. We therefore get the greatest speedup for large system sizes at low temperature where many prospective transitions are rejected at the classical stage and the matrix computation takes up the greatest fraction of the total computation time. The upshot is that we find a speedup of up to a factor of 10 at the cost of very little extra algorithmic complexity.

Our two-step method should be distinguished from the more common method for speeding up MCMC which is to add asymmetry to the proposal distribution to make it as similar as possible to $\min\left(1, e^{-\beta \Delta E}\right)$. This reduces the number of rejected states, which brings the algorithm closer in efficiency to a direct sampling method. However it comes at the expense of requiring a way to directly sample from this complex distribution, a problem which MCMC was employed to solve in the first place. For example, recent work trains restricted Boltzmann machines (RBMs) to generate samples for the proposal distribution of the FK model [@huangAcceleratedMonteCarlo2017]. The RBMs are chosen as a parametrisation of the proposal distribution that can be efficiently sampled from while offering sufficient flexibility that they can be adjusted to match the target distribution. Our proposed method is considerably simpler and does not require training while still reaping some of the benefits of reduced computation.

# []{#app:balance label="app:balance"} DETAILED BALANCE 

Given a MCMC algorithm with target distribution $\pi(a)$ and transition function $\T$ the detailed balance condition is sufficient (along with some technical constraints [@wolffMonteCarloErrors2004]) to guarantee that in the long time limit the algorithm produces samples from $\pi$. $$\pi(a)\T(a \to b) = \pi(b)\T(b \to a)$$

In pseudo-code, our two step method corresponds to two nested comparisons with the majority of the work only occurring if the first test passes:

``` python
current_state = initial_state

for i in range(N_steps):
  new_state = proposal(current_state)

  c_dE = classical_energy_change(
                               current_state,
                               new_state)
  if uniform(0,1) < exp(-beta * c_dE):
    q_dF = quantum_free_energy_change(
                                current_state,
                                new_state)
    if uniform(0,1) < exp(- beta * q_dF):
      current_state = new_state

    states[i] = current_state
```

Defining $r_c = e^{-\beta H_c}$ and $r_q = e^{-\beta F_q}$ our target distribution is $\pi(a) = r_c r_q$. This method has $\T(a\to b) = q(a\to b)\A(a \to b)$ with symmetric $p(a \to b) = \p(b \to a)$ and $\A = \min\left(1, r_c\right) \min\left(1, r_q\right)$

Substituting this into the detailed balance equation gives: $$\T(a \to b)/\T(b \to a) = \pi(b)/\pi(a) = r_c r_q$$

Taking the LHS and substituting in our transition function: $$\begin{aligned}
\T(a \to b)/\T(b \to a) = \frac{\min\left(1, r_c\right) \min\left(1, r_q\right)}{ \min\left(1, 1/r_c\right) \min\left(1, 1/r_q\right)}\end{aligned}$$

which simplifies to $r_c r_q$ as $\min(1,r)/\min(1,1/r) = r$ for $r > 0$.

### Markov Chain Monte Carlo

To evaluate thermodynamic averages we perform a classical Markov Chain Monte Carlo random walk over the space of ionic configurations, at each step diagonalising the effective electronic Hamiltonian @maskaThermodynamicsTwodimensionalFalicovKimball2006}. Using a binder-cumulant method [@binderFiniteSizeScaling1981; @musialMonteCarloSimulations2002], we demonstrate the model has a finite temperature phase transition when the interaction is sufficiently long ranged. We then estimate the density of states and the inverse participation ratio as a function of energy to diagnose localisation properties. We show preliminary results that the in-gap states induced at finite temperature are localised while the states in the unperturbed bands remain extended, evidence for a mobility edge.

### Diagnostics of Localisation
#### Inverse Participation Ratio
The inverse participation ratio is defined for a normalised wave function $\psi_i = \psi(x_i), \sum_i \abs{\psi_i}^2 = 1$ as its fourth moment [@kramerLocalizationTheoryExperiment1993]:
$$
P^{-1} = \sum_i \abs{\psi_i}^4
$$
%
It acts as a measure of the portion of space occupied by the wave function. For localised states it will be independent of system size while for plane wave states in d dimensions $P = L^d $. States may also be intermediate between localised and extended, described by their fractal dimensionality $d > d* > 0$:
$$
P(L) \goeslike L^{d*} 
$$
%
For extended states $d* = 0$ while for localised ones $d* = 0$. In this work we take use an energy resolved IPR [@andersonAbsenceDiffusionCertain1958]:
$$
DOS(\omega) = \sum_n \delta(\omega - \epsilon_n)
IPR(\omega) = DOS(\omega)^{-1} \sum_{n,i} \delta(\omega - \epsilon_n) \abs{\psi_{n,i}}^4
$$
Where $\psi_{n,i}$ is the wavefunction corresponding to the energy $\epsilon_n$ at the ith site. In practice we bin the energies and IPRs into a fine energy grid and use Lorentzian smoothing if necessary. 

#### Transfer Matrix Approach
The transfer matrix method (TMM) can be used to calculate the localisation length of the eigenstates of a system. Following Refs. [@kramerLocalizationTheoryExperiment1993; @smithDynamicalLocalizationMathbbZ2018], for bi-linear, 1D Hamiltonians the method represents the action of $H$ on a state $\ket{\psi} = \sum_i \psi_i \ket{i}$ with energy E by a matrix equation:
$$
H &= - \sum_i (c^\dagger_i c_{i+1} + c^\dagger_{i+1} c_{i}) - \sum_i h_i c^\dagger_i c_i \\
E\ket{\psi} &= H \ket{\psi} \\
\label{eq:tmm_difference} E \psi_i &= -(\psi_{i+1} + \psi_{i-1}) - h_i \psi_i 
$$
%
Where Eq. \ref{eq:tmm_difference} can be represented by a matrix equation:
$$
\begin{pmatrix}
\psi_{i+1}\\
\psi_{i}
\end{pmatrix}
=
\begin{pmatrix}
-(E + h_i) &  -1\\
1 & 0
\end{pmatrix}
\begin{pmatrix}
\psi_{i}\\
\psi_{i-1}
\end{pmatrix}
= T_i 
\begin{pmatrix}
\psi_{i}\\
\psi_{i-1}
\end{pmatrix}
$$
%
Defining a product along the chain:
$$Q_L = \prod_{i=0}^L T_i$$
%
Oseledec’s theorem proves that there exists a limiting matrix $\Gamma$:
$$
\Gamma = \lim_{L \to \infty} (Q_L Q_L^\dagger)^{\frac{1}{2L}}
$$
%
with eigenvalues $\exp(\gamma_i)$ where $\gamma_i$ are the Lyapunov exponents of $Q_L$. The smallest Lyapunov exponent is the inverse of the localisation length of the state. In practice one takes $Q_L$ with L equal to the system size, finds the smallest eigenvalue q and estimates the localisation length by:
$$
\lambda = \frac{L}{\ln{q}}
$$
%
As noted by [@smithDynamicalLocalizationMathbbZ2018] this method can be numerically unstable because the matrix elements diverge or vanish exponentially. To get around this, the authors break the matrix multiplication into chunks and the logarithms of the eigenvalues of each are stored separately.

## Numerical Methods
In this section we will define the Markov Chain Monte Carlo (MCMC) method in general then detail its application to the FK model. We will then cover methods applicable to the measurements obtained from MCMC. These include calculation of the density of states and energy resolved inverse participation ratio as well as phase transition diagnostics such as the Binder cumulant.

### Markov Chain Monte Carlo
Markov Chain Monte Carlo (MCMC) is a technique for evaluating thermal expectation values $\expval{O}$ with respect to some physical system defined by a set of states $\{x: x \in S\}$ and a free energy $F(x)$ [@krauthIntroductionMonteCarlo1998]. The thermal expectation value is defined via a Boltzmann weighted sum over the entire states:
$$
    \tex{O} &= \frac{1}{\Z} \sum_{x \in S} O(x) P(x) \\
    P(x) &= \frac{1}{\Z} e^{-\beta F(x)} \\
    \Z &= \sum_{x \in S} e^{-\beta F(x)}
$$

 When the state space is too large to evaluate this sum directly, MCMC defines a stochastic algorithm which generates a random walk $\{x_0\ldots x_i\ldots x_N\}$ whose distribution $p(x_i)$ approaches a target distribution $P(x)$ in the large N limit. 
 
 $$\lim_{i\to\infty} p(x_i) = P(x)$$

In this case the target distribution will be the thermal one $P(x) \rightarrow \Z^{-1} e^{-\beta F(x)}$. The major benefit of the method being that as long as one can express the desired $P(x)$ up to a multiplicative constant, MCMC can be applied. Since $e^{-\beta F(x)}$ is relatively easy to evaluate, MCMC provides a useful method for finite temperature physics. 

Once the random walk has been carried out for many steps, the expectation values of $O$ can be estimated from the MCMC samples:
$$
    \tex{O} = \sum_{i = 0}^{N} O(x_i) + \mathcal{O}(\frac{1}{\sqrt{N}})
$$
The the samples in the random walk are correlated so the samples effectively contain less information than $N$ independent samples would. As a consequence the variance is larger than the $\tex{O^2} - \tex{O}^2$ form it would have if the estimates were uncorrelated. Methods of estimating the true variance of $\tex{O}$ and decided how many steps are needed will be considered later.  

## The Metropolis-Hasting Algorithm

Markov chains are defined by a transition function $\T(x_{i+1} \rightarrow x_i) $ giving the probability that a chain in state $x_i$ at time $i$ will transition to a state $x_{i+1}$. Since we must transition somewhere at each step, this comes with the normalisation condition that $\sum\limits_x \T(x' \rightarrow x) = 1$. 

If we define an ensemble of Markov chains and consider the distributions we get a sequence of distributions $\{p_0(x), p_1(x),  p_2(x)\ldots\}$ with 
$$p_{i+1}(x) &= \sum_{x' \in S} p_i(x') \T(x' \rightarrow x)$$
$p_o(x)$ might be a delta function on one particular starting state which would then be smoothed out by the transition function repeatedly.

As we'd like to draw samples from the target distribution $P(x)$ the trick is to choose $\T(x_{i+1} \rightarrow x_i) $ such that :

$$
P(x) &= \sum_{x'} P(x') \T(x' \rightarrow x)
$$
In other words the MCMC dynamics defined by $\T$ must be constructed to have the target distribution as their only fixed point. This condition is called the global balance condition. Along with some more technical considerations such as ergodicity which won't be considered here, global balance suffices to ensure that a MCMC method is correct. 

A sufficient but not necessary condition for global balance to hold is detailed balance:

$$
P(x) \T(x \rightarrow x') = P(x') \T(x' \rightarrow x)
$$
%
In practice most algorithms are constructed to satisfy detailed balance though there are arguments that relaxing the condition can lead to faster algorithms [@kapferSamplingPolytopeHarddisk2013].

The goal of MCMC is then to choose $\T$ so that it has the desired thermal distribution $P(x)$ as its fixed point and that it converges quickly onto it. This boils down to requiring that the matrix representation of $T_{ij} = \T(x_i \to x_j)$ has an eigenvector equal to $P_i = P(x_i)$ with eigenvalue 1 and all other eigenvalues with magnitude less than one. The convergence time depends on the magnitude of the second largest eigenvalue.

## Metropolis-Hastings
In order to actually choose new states according to $\T$ one chooses states from a proposal distribution $q(x_i \to x')$ that can be directly sampled from. For instance, this might mean flipping a single random spin in a spin chain, in which case $q(x'\to x_i)$ is the uniform distribution on states reachable by one spin flip from $x_i$. The proposal $x'$ is then accepted or rejected with an acceptance probability $\A(x'\to x_{i+1})$, if the proposal is rejected then $x_{i+1} = x_{i}$. Now $\T(x\to x') = q(x\to x')\A(x \to x')$.

The Metropolis-Hasting algorithm is a slight extension of the original Metropolis algorithm that allows for non-symmetric proposal distributions $q(x\to x') \neq q(x'\to x) $. It can be derived starting from detailed balance [@krauthIntroductionMonteCarlo1998]:
$$
P(x)\T(x \to x') &= P(x')\T(x' \to x) \\
P(x)q(x \to x')\A(x \to x') &= P(x')q(x' \to x)\A(x' \to x) \\
\label{eq:db2} \frac{\A(x \to x')}{\A(x' \to x)} &= \frac{P(x')q(x' \to x)}{P(x)q(x \to x')} = f(x, x')\\
$$
%
The Metropolis-Hastings algorithm is the choice:
$$\label{eq:mh} \A(x \to x') = \min\left(1, f(x,x')\right)$$
%
Noting that $f(x,x') = 1/f(x',x)$, Eq. \ref{eq:mh} can be seen to satisfy Eq. \ref{eq:db2} by considering the two cases $f(x,x') > 1$ and $f(x,x') < 1$.

By choosing the proposal distribution such that $f(x,x')$ is as close as possible to one, the rate of rejections can be reduced and the algorithm sped up.
    
## Convergence, Auto-correlation and Binning
%Thinning, burn in, multiple runs
%Simulated annealing and Parallel Tempering

# Applying MCMC to the FK model
MCMC can be applied to sample over the classical degrees of freedom of the model. We take the full Hamiltonian and split it into a classical and a quantum part:
$$
    H_{\mathrm{FK}} &= -\sum_{<ij>} c^\dagger_{i}c_{j} + U \sum_{i} (c^\dagger_{i}c_{i} - 1/2)( n_i - 1/2) \\
    &+ \sum_{ij} J_{ij} (n_i - 1/2) (n_j - 1/2)  - \mu \sum_i (c^\dagger_{i}c_{i} + n_i)\\
    H_q &= -\sum_{<ij>} c^\dagger_{i}c_{j} + \sum_{i} \left(U(n_i - 1/2) - \mu\right) c^\dagger_{i}c_{i}\\
    H_c &= \sum_i \mu n_i - \frac{U}{2}(n_i - 1/2) + \sum_{ij}J_{ij}(n_i - 1/2)(n_j - 1/2)
$$
%
There are $2^N$ possible ion configurations $\{ n_i \}$, we define $n^k_i$ to be the occupation of the ith site of the kth configuration. The quantum part of the free energy can then be defined through the quantum partition function $\Z^k$ associated with each ionic state $n^k_i$:
$$
F^k &= -1/\beta \ln{\Z^k} \\
$$
%
Such that the overall partition function is:
$$
\Z &= \sum_k e^{- \beta H^k} Z^k \\
&= \sum_k e^{-\beta (H^k + F^k)} \\
$$
%
Because fermions are limited to occupation numbers of 0 or 1 $Z^k$ simplifies nicely. If $m^j_i = \{0,1\}$ is defined as the occupation of the level with energy $\epsilon^k_i$ then the partition function is a sum over all the occupation states labelled by j:
$$
Z^k    &= \Tr e^{-\beta F^k} = \sum_j e^{-\beta \sum_i m^j_i \epsilon^k_i}\\
       &= \sum_j \prod_i e^{- \beta m^j_i \epsilon^k_i}= \prod_i \sum_j e^{- \beta m^j_i \epsilon^k_i}\\
       &= \prod_i (1 + e^{- \beta \epsilon^k_i})\\
F^k    &= -1/\beta \sum_k \ln{(1 + e^{- \beta \epsilon^k_i})}
$$
%
Observables can then be calculated from the partition function, for examples the occupation numbers:

$$
\tex{N} &= \frac{1}{\beta} \frac{1}{Z} \frac{\partial Z}{\partial \mu} = - \frac{\partial F}{\partial \mu}\\
    &= \frac{1}{\beta} \frac{1}{Z} \frac{\partial}{\partial \mu} \sum_k e^{-\beta (H^k + F^k)}\\
    &= 1/Z \sum_k (N^k_{\mathrm{ion}} + N^k_{\mathrm{electron}}) e^{-\beta (H^k + F^k)}\\
$$
%
with the definitions:

$$
N^k_{\mathrm{ion}} &= - \frac{\partial H^k}{\partial \mu} = \sum_i n^k_i\\
N^k_{\mathrm{electron}} &= - \frac{\partial F^k}{\partial \mu} = \sum_i \left(1 + e^{\beta \epsilon^k_i}\right)^{-1}\\
$$
%
The MCMC algorithm consists of performing a random walk over the states $\{ n^k_i \}$. In the simplest case the proposal distribution corresponds to flipping a random site from occupied to unoccupied or vice versa, since this proposal is symmetric the acceptance function becomes:
$$ 
P(k) &= \Z^{-1} e^{-\beta(H^k + F^k)} \\
\A(k \to k') &= \min\left(1, \frac{P(k')}{P(k)}\right) = \min\left(1, e^{\beta(H^{k'} + F^{k'})-\beta(H^k + F^k)}\right)
$$
%
At each step $F^k$ is calculated by diagonalising the tri-diagonal matrix representation of $H_q$ with open boundary conditions. Observables are simply averages over the their value at each step of the random walk. The full spectrum and eigenbasis is too large to save to disk so usually running averages of key observables are taken as the walk progresses.

## Proposal Distributions

In a MCMC method a key property is the proportion of the time that proposals are accepted, the acceptance rate. If this rate is too low the random walk is trying to take overly large steps in energy space which problematic because it means very few new samples will be generated. If it is too high it implies the steps are too small, a problem because then the walk will take longer to explore the state space and the samples will be highly correlated. Ideal values for the acceptance rate can be calculated under certain assumptions [@robertsWeakConvergenceOptimal1997]. Here we monitor the acceptance rate and if it is too high we re-run the MCMC with a modified proposal distribution that has a chance to propose moves that flip multiple sites at a time. 

In addition we exploit the particle-hole symmetry of the problem by occasionally proposing a flip of the entire state. This works because near half-filling, flipping the occupations of all the sites will produce a state at or near the energy of the current one.

## Perturbation MCMC
The matrix diagonalisation is the most computationally expensive step of the process, a speed up can be obtained by modifying the proposal distribution to depend on the classical part of the energy, a trick gleaned from Ref. [@krauthIntroductionMonteCarlo1998]:
$$
q(k \to k') &= \min\left(1, e^{\beta (H^{k'} - H^k)}\right) \\
\A(k \to k') &= \min\left(1, e^{\beta(F^{k'}- F^k)}\right)
$$
%
This allows the method to reject some states without performing the diagonalisation at no cost to the accuracy of the MCMC method. 

An extension of this idea is to try to define a classical model with a similar free energy dependence on the classical state as the full quantum, Ref. [@huangAcceleratedMonteCarlo2017] does this with restricted Boltzmann machines whose form is very similar to a classical spin model.

## Scaling
In order to reduce the effects of the boundary conditions and the finite size of the system we redefine and normalise the coupling matrix to have 0 derivative at its furthest extent rather than cutting off abruptly.

$$
J'(x) &= \abs{\frac{L}{\pi}\sin \frac{\pi x}{L}}^{-\alpha} \\
J(x) &= \frac{J_0 J'(x)}{\sum_y J'(y)}
$$
%
The scaling ensures that, in the ordered phase, the overall potential felt by each site due to the rest of the system is independent of system size.

## Binder Cumulants
The Binder cumulant is defined as:
$$U_B = 1 - \frac{\tex{\mu_4}}{3\tex{\mu_2}^2}$$ 
%
where 
$$\mu_n = \tex{(m - \tex{m})^n}$$ 
%
are the central moments of the order parameter m:
$$m = \sum_i (-1)^i (2n_i - 1) / N$$
%
The Binder cumulant evaluated against temperature can be used as a diagnostic for the existence of a phase transition. If multiple such curves are plotted for different system sizes, a crossing indicates the location of a critical point [@binderFiniteSizeScaling1981; @musialMonteCarloSimulations2002].


## Markov Chain Monte-Carlo in Practice

### Quick Intro to MCMC}

The main paper relies on \ac{MCMC} extensively to evaluate thermal expectation values within the \ac{FK} model by walking over states of the classical spin system $S_i$. For a classical system, the thermal expectation value of some operator $O$ is defined by a Boltzmann weighted sum over the classical state space:
$$
    \tex{O} &= \frac{1}{\Z} \sum_{\s \in S} O(x) P(x) \\
    P(x) &= \frac{1}{\Z} e^{-\beta F(x)} \\
    \Z &= \sum_{\s \in S} e^{-\beta F(x)}
$$
While for a quantum system these sums are replaced by equivalent traces. The obvious approach to evaluate these sums numerically would be to directly loop over all the classical states in the system and perform the sum. But we all know know why this isn't feasible: the state space is too large! Indeed even if we could do it, it would still be computationally wasteful since at low temperatures the sums are dominated by low energy excitations about the ground states of the system. Even worse, in our case we must fully solve the fermionic system via exact diagonalisation for each classical state in the sum, a very expensive operation!~\footnote{The effort involved in exact diagonalisation scales like $N^2$ for systems with a tri-diagonal matrix representation (open boundary conditions and nearest neighbour hopping) and like $N^3$ for a generic matrix [@bolchQueueingNetworksMarkov2006; @usmaniInversionTridiagonalJacobi1994].

\clearpage
\begin{figure}
  \centering
  \includegraphics[width=\columnwidth]{pdf_figs/raw_steps_single_flip.pdf}
  \caption{An MCMC walk starting from the staggered charge density wave ground state for a system with $N = 100$ sites and 10,000 MCMC steps. In this simulation only a single spin can be flipped per step according to the Metropolis-Hastings Algorithm. The staggered magnetisation $m = N^{-1} \sum_i (-1)^i \; S_i$ order parameter is plotted below. At this temperature the thermal average of m is zero, while the initial state has m = 1. We see that it takes about 1000 steps for the system to converge, after which it moves about the m = 0 average with a finite auto-correlation time.  $t = 1, \alpha = 1.25, T = 3, J = U = 5 $ \label{fig:raw}}
\end{figure}

\ac{MCMC} sidesteps these issues by defining a random walk that focuses on the states with the greatest Boltzmann weight. At low temperatures this means we need only visit a few low energy states to make good estimates while at high temperatures the weights become uniform so a small number of samples distributed across the state space suffice. However we will see that the method is not without difficulties of its own.

\begin{figure}
  \centering
  \includegraphics[width=\columnwidth]{pdf_figs/single.pdf}
  \caption{Two MCMC chains starting from the same initial state for a system with $N = 90$ sites and 1000 MCMC steps.  In this simulation the MCMC step is defined differently: an attempt is made to flip n spins, where n is drawn from Uniform(1,N). This is repeated $N^2/100$ times for each step. This trades off computation time for storage space, as it makes the samples less correlated, giving smaller statistical error for a given number of stored samples. These simulations therefore have the potential to necessitate $N^2/100$ matrix diagonalisations for every MCMC sample, though this can be cut down with caching and other tricks. $t = 1, \alpha = 1.25, T = 2.2, J = U = 5 $ \label{fig:single}}
\end{figure}

%MCMC from an ensemble point of view
In implementation \ac{MCMC} can be boiled down to choosing a transition function $\T(\s_{t} \rightarrow \s_t+1) $ where $\s$ are vectors representing classical spin configurations. We start in some initial state $\s_0$ and then repeatedly jump to new states according to the probabilities given by $\T$. This defines a set of random walks $\{\s_0\ldots \s_i\ldots \s_N\}$. Fig.~\ref{fig:single} shows this in practice: we have a (rather small) ensemble of $M = 2$ walkers starting at the same point in state space and then spreading outwards by flipping spins along the way. 

In pseudo-code one could write the MCMC simulation for a single walker as:

\begin{lstlisting}[language=Python]
current_state = initial_state

for i in range(N_steps):
    new_state = sample_T(current_state) 
    states[i] = current_state
\end{lstlisting}

Where the \texttt{sample\_T} function here produces a state with probability determined by the \texttt{current\_state} and the transition function $\T$.

If we ran many such walkers in parallel we could then approximate the distribution $p_t(\s; \s_0)$ which tells us where the walkers are likely to be after they've evolved for $t$ steps from an initial state $\s_0$. We need to carefully choose $\T$ such that after a large number of steps $k$ (the convergence time) the probability $p_t(\s;\s_0)$ approaches the thermal distribution $P(\s; \beta) = \Z^{-1} e^{-\beta F(\s)}$. This turns out to be quite easy to achieve using the Metropolis-Hasting algorithm.

### Convergence Time}

Considering $p(\s)$ as a vector $\vec{p}$ whose jth entry is the probability of the jth state $p_j = p(\s_j)$, and writing $\T$ as the matrix with entries $T_{ij} = \T(\s_j \rightarrow \s_i)$ we can write the update rule for the ensemble probability as:
$$\vec{p}_{t+1} = \T \vec{p}_t \implies \vec{p}_{t} = \T^t \vec{p}_0$$
where $\vec{p}_0$ is vector which is one on the starting state and zero everywhere else. Since all states must transition to somewhere with probability one: $\sum_i T_{ij} = 1$. 

Matrices that satisfy this are called stochastic matrices exactly because they model these kinds of Markov processes. It can be shown that they have real eigenvalues, and ordering them by magnitude, that $\lambda_0 = 1$ and $0 < \lambda_{i\neq0} < 1$. %https://en.wikipedia.org/wiki/Stochastic_matrix
Assuming $\T$ has been chosen correctly, its single eigenvector with eigenvalue 1 will be the thermal distribution \footnote{or, in the general case, any desired distribution. MCMC has found a lot of use in sampling from the complicated distributions that arise when taking a Bayesian approach to statistics.} so repeated application of the transition function eventually leads there, while memory of the initial conditions decays exponentially with a convergence time $k$ determined by $\lambda_1$. In practice this means that one throws away the data from the beginning of the random walk in order reduce the dependence on the initial conditions and be close enough to the target distribution. 

### Auto-correlation Time}


\begin{figure}
  \centering
  \includegraphics[width=\columnwidth]{figs/m_autocorr.png}
  \caption{(Upper) 10 MCMC chains starting from the same initial state for a system with $N = 150$ sites and 3000 MCMC steps. At each MCMC step, n spins are flipped where n is drawn from Uniform(1,N) and this is repeated $N^2/100$ times. The simulations therefore have the potential to necessitate $10*N^2$ matrix diagonalisations for each 100 MCMC steps. (Lower) The normalised auto-correlation $(\expval{m_i m_{i-j}} - \expval{m_i}\expval{m_i}) / Var(m_i))$ averaged over $i$. It can be seen that even with each MCMC step already being composed of many individual flip attempts, the auto-correlation is still non negligible and must be taken into account in the statistics. $t = 1, \alpha = 1.25, T = 2.2, J = U = 5 $ \label{fig:m_autocorr}}
\end{figure}

At this stage one might think we're done. We can indeed draw independent samples from $P(\s; \beta)$ by starting from some arbitrary initial state and doing $k$ steps to arrive at a sample. However a key insight is that after the convergence time, every state generated  is a sample from $P(\s; \beta)$! They are not, however, independent samples. In Fig.~\ref{fig:raw} it is already clear that the samples of the order parameter m have some auto-correlation because only a few spins are flipped each step but even when the number of spins flipped per step is increased, Fig.~\ref{fig:m_autocorr} shows that it can be an important effect near the phase transition. Let's define the auto-correlation time $\tau(O)$ informally as the number of MCMC samples of some observable O that are statistically equal to one independent sample or equivalently as the number of MCMC steps after which the samples are correlated below some cutoff, see [@krauthIntroductionMonteCarlo1996] for a more rigorous definition involving a sum over the auto-correlation function. The auto-correlation time is generally shorter than the convergence time so it therefore makes sense from an efficiency standpoint to run a single walker for many MCMC steps rather than to run a huge ensemble for $k$ steps each. 

Once the random walk has been carried out for many steps, the expectation values of $O$ can be estimated from the MCMC samples $\s_i$:
$$
    \tex{O} = \sum_{i = 0}^{N} O(\s_i) + \mathcal{O}(\frac{1}{\sqrt{N}})
$$
The the samples are correlated so the N of them effectively contains less information than $N$ independent samples would, in fact roughly $N/\tau$ effective samples. As a consequence the variance is larger than the $\qex{O^2} - \qex{O}^2$ form it would have if the estimates were uncorrelated. There are many methods in the literature for estimating the true variance of $\qex{O}$ and deciding how many steps are needed but my approach has been to run a small number of parallel chains, which are independent, in order to estimate the statistical error produced. This is a slightly less computationally efficient because it requires throwing away those $k$ steps generated before convergence multiple times but it is a conceptually simple workaround.

In summary, to do efficient simulations we want to reduce both the convergence time and the auto-correlation time as much as possible. In order to explain how, we need to introduce the Metropolis-Hasting (MH) algorithm and how it gives an explicit form for the transition function. 

### The Metropolis-Hastings Algorithm}

MH breaks up the transition function into a proposal distribution $q(\s \to \s')$ and an acceptance function $\A(\s \to \s')$. $q$ needs to be something that we can directly sample from, and in our case generally takes the form of flipping some number of spins in $\s$, i.e if we're flipping a single random spin in the spin chain, $q(\s \to \s')$ is the uniform distribution on states reachable by one spin flip from $\s$. This also gives the nice symmetry property that $q(\s \to \s') = q(\s' \to \s)$. 

The proposal $\s'$ is then accepted or rejected with an acceptance probability $\A(\s \to \s')$, if the proposal is rejected then $\s_{i+1} = \s_{i}$. Hence:

$$\T(x\to x') = q(x\to x')\A(x \to x')$$

When the proposal distribution is symmetric as ours is, it cancels out in the expression for the acceptance function and the Metropolis-Hastings algorithm is simply the choice:
$$ \A(x \to x') = \min\left(1, e^{-\beta\;\Delta F}\right)$$
Where $F$ is the overall free energy of the system, including both the quantum and classical sector.

To implement the acceptance function in practice we pick a random number in the unit interval and accept if it is less than $e^{-\beta\;\Delta F}$:

\begin{lstlisting}[language=Python]
current_state = initial_state

for i in range(N_steps):
    new_state = proposal(current_state)
    df = free_energy_change(current_state, new_state, parameters)

    if uniform(0,1) < exp(-beta * df):
        current_state = new_state
        
    states[i] = current_state
\end{lstlisting}

This has the effect of always accepting proposed states that are lower in energy and sometimes accepting those that are higher in energy than the current state. 

### Choosing the proposal distribution}
\begin{figure}[H]
  \centering
  \includegraphics[width=\columnwidth]{figs/autocorr_multiple_proposals.png}
  Simulations showing how the autocorrelation of the order parameter depends on the proposal distribution used at different temperatures, we see that at $T = 1.5 < T_c$ a single spin flip is likely the best choice, while at the high temperature $T = 2.5 > T_c$ flipping two sites or a mixture of flipping two and 1 sites is likely a better choice. 
  \caption{ $t = 1, \alpha = 1.25, J = U = 5 $ \label{fig:comparison}}
\end{figure}

Now we can discuss how to minimise the auto-correlations. The general principle is that one must balance the proposal distribution between two extremes. Choose overlay small steps, like flipping only a single spin and the acceptance rate will be high because $\Delta F$ will usually be small, but each state will be very similar to the previous and the auto-correlations will be high too, making sampling inefficient. On the other hand, overlay large steps, like randomising a large portion of the spins each step, will result in very frequent rejections, especially at low temperatures. 

I evaluated a few different proposal distributions for use with the FK model. 

\begin{enumerate}
\item Flipping a single random site
\item Flipping N random sites for some N
\item Choosing n from Uniform(1, N) and then flipping n sites for some fixed N.
\item Attempting to tune the proposal distribution for each parameter regime.
\end{enumerate}

Fro Figure~\ref{fig:comparison} we see that even at moderately high temperatures $T > T_c$ flipping one or two sites is the best choice. However for some simulations at very high temperature flipping more spins is warranted. Tuning the proposal distribution automatically seems like something that would not yield enough benefit for the additional complexity it would require.

### Two Step Trick

Our method already relies heavily on the split between the classical and quantum sector to derive a sign problem free MCMC algorithm but it turns out that there is a further trick we can play with it. The free energy term is the sum of an easy to compute classical energy and a more expensive quantum free energy, we can split the acceptance function into two in such as way as to avoid having to compute the full exact diagonalisation some of the time:

```python

current_state = initial_state

for i in range(N_steps):
    new_state = proposal(current_state)

    df_classical = classical_free_energy_change(current_state, new_state, parameters)
    if exp(-beta * df_classical) < uniform(0,1):
        f_quantum = quantum_free_energy(current_state, new_state, parameters)
    
        if exp(- beta * df_quantum) < uniform(0,1):
          current_state = new_state
    
        states[i] = current_state
    
```